In [3]:
import os
import sys
import glob
import numpy as np
import pandas as pd
from IPython.display import display
import time 
import tracker
latest_date = tracker.processing_dates[-1]
os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/kfetter/hichip-db-loop-calling/')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth', 400)

In [4]:
latest_date

'2022.08.18.08.42'

In [5]:
# setting input and output with jupyter notebook in context 
if 'ipykernel_launcher.py' in sys.argv[0]:
    input_fn = 'results/samplesheets/hicpro/{}.hicpro.samplesheet.with_header.tsv'.format(latest_date)
    output_prefix = 'results/samplesheets/post-hicpro/{}.post-hicpro.post-check.fithichip-peaks'.format(latest_date)
else:
    input_fn = sys.argv[1]
    output_fn = sys.argv[2]

## Helper Functions

In [6]:
def read_out_log(log):
    
    status = 0
    sample_name = 'check'
    date = ''

    # extract job_id and index
    meta = os.path.basename(log)
    meta = meta.split('.')[1].split('-')
    job_id = meta[0].replace('o', '')
    index = meta[1]

    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'sample_name' in info:
                sample_name = info.split()[1]
            elif '# Ended: hichip-peaks' in info:
                status = 1

    return([index, sample_name, job_id, status])

In [7]:
def read_error_log(log):
    
    status = 0
    
    with open(log, errors='ignore') as fr:
        for line in fr:
            info = line.strip()
            if 'INFO - Done!' in info:
                status = 1
    
    return(status)

## Check Logs

In [8]:
# load the data
df = pd.read_table(input_fn)
df['sample_index'] = df.index + 1 # add 1 since we'll be using 1-based indexing with sed

In [9]:
df

,std_sample_name,gse_id,organism,bio_rep,antibody_target,restriction_enzyme,sample_name,sample_index
0,3134_siCTRL_1hr_Dex.GSE162617.Mus_Musculus.GR.b1,GSE162617,Mus_Musculus,1,GR,MboI,3134_siCTRL_1hr_Dex,1
1,mES_25m.GSE101498.Mus_Musculus.H3K27ac.b1,GSE101498,Mus_Musculus,1,H3K27ac,MboI,mES_25m,2
2,3134_siNIPBL_1hr_Dex.GSE162617.Mus_Musculus.GR.b1,GSE162617,Mus_Musculus,1,GR,MboI,3134_siNIPBL_1hr_Dex,3
3,mES_25m.GSE101498.Mus_Musculus.H3K27ac.b2,GSE101498,Mus_Musculus,2,H3K27ac,MboI,mES_25m,4
4,3134_WT.GSE162617.Mus_Musculus.GR.b1,GSE162617,Mus_Musculus,1,GR,MboI,3134_WT,5
5,3T3.GSE192387.Mus_Musculus.BATF.b1,GSE192387,Mus_Musculus,1,BATF,MboI,3T3,6
6,3T3.GSE192387.Mus_Musculus.CTCF.b1,GSE192387,Mus_Musculus,1,CTCF,MboI,3T3,7
7,3T3_Irf4.GSE192387.Mus_Musculus.BATF.b1,GSE192387,Mus_Musculus,1,BATF,MboI,3T3_Irf4,8
8,3T3_Irf4_Runx3_Tbet.GSE192387.Mus_Musculus.BATF.b1,GSE192387,Mus_Musculus,1,BATF,MboI,3T3_Irf4_Runx3_Tbet,9
9,3T3_Norm.GSE178344.Mus_Musculus.SMC1A.b1,GSE178344,Mus_Musculus,1,SMC1A,MboI,3T3_Norm,10


In [10]:
print('number of samples:', len(df))

number of samples: 192


In [11]:
# create a dataframe of log information
output_logs = glob.glob('results/peaks/logs/run_hichip_peaks.o*-*')
log_data = []   
for out_log in output_logs: 
    sample_index = out_log.split('-')[1]
    index, sample_name, job_id, out_status = read_out_log(out_log)
    error_log = [log for log in glob.glob('results/peaks/logs/run_hichip_peaks.e*-*') if log.split('-')[1] == sample_index][0]
    error_status = read_error_log(error_log)
    log_data.append([index, sample_name, job_id, out_status, error_status, os.path.basename(out_log)])
log_df = pd.DataFrame(log_data)
log_df.columns = ['sample_index', 'std_sample_name', 'job_id', 'out_status', 'error_status', 'log']
log_df.drop('sample_index', inplace=True, axis=1)

In [12]:
log_df = log_df.sort_values(by=["std_sample_name"], ascending=True).reset_index(drop=True)
log_df

,std_sample_name,job_id,out_status,error_status,log
0,3134_WT.GSE162617.Mus_Musculus.GR.b1,5147938,1,1,run_hichip_peaks.o5147938-5
1,3134_siCTRL_1hr_Dex.GSE162617.Mus_Musculus.GR.b1,5147929,1,1,run_hichip_peaks.o5147929-1
2,3134_siNIPBL_1hr_Dex.GSE162617.Mus_Musculus.GR.b1,5147938,1,1,run_hichip_peaks.o5147938-3
3,3T3.GSE192387.Mus_Musculus.BATF.b1,5147938,1,1,run_hichip_peaks.o5147938-6
4,3T3.GSE192387.Mus_Musculus.CTCF.b1,5147938,1,1,run_hichip_peaks.o5147938-7
5,3T3_Irf4.GSE192387.Mus_Musculus.BATF.b1,5147938,1,1,run_hichip_peaks.o5147938-8
6,3T3_Irf4_Runx3_Tbet.GSE192387.Mus_Musculus.BATF.b1,5147938,1,1,run_hichip_peaks.o5147938-9
7,3T3_Norm.GSE178344.Mus_Musculus.SMC1A.b1,5147938,1,1,run_hichip_peaks.o5147938-10
8,3T3_TCF1.GSE178344.Mus_Musculus.SMC1A.b1,5147938,1,1,run_hichip_peaks.o5147938-11
9,AML12_shCtrl.GSE141113.Mus_Musculus.H3K9me3.b1,5147938,1,1,run_hichip_peaks.o5147938-20


## Identifying problem samples

In [13]:
problems = (log_df.out_status != 1) | (log_df.error_status != 1)
problems_df = log_df.loc[problems,:]
print('nubmer of problem samples:', len(problems_df))

nubmer of problem samples: 0


In [14]:
problems_df.loc[:, ['std_sample_name', 'job_id', 'out_status', 'out_status', 'log']]

,std_sample_name,job_id,out_status,out_status,log


In [17]:
header_output = '{}.with_header.tsv'.format(output_prefix)
problems_df.to_csv(header_output, header=True, index=False, sep='\t')